In [3]:
from typing import get_args, Type
from dataclasses import dataclass, fields
import math
import numpy as np
import pygmo as pg

# Try to see when it breaks
from solarmed_optimization import (DecisionVariablesUpdates, 
                                   DecisionVariables,
                                   OptimVarIdstoModelVarIdsMapping)
import pandas as pd
from solarmed_modeling.solar_med import SolarMED
from solarmed_modeling.fsms import SolarMedState

pop_size: int = 10

def fitness(x: np.ndarray | int) -> np.ndarray[float] | list[float]:
    obj = 0
    for i in range(3):
        obj += (x[2*i-2]-3)**2 / 1000. - (x[2*i-2]-x[2*i-1]) + math.exp(20.*(x[2*i - 2]-x[2*i-1]))

    ce1 = 4*(x[0]-x[1])**2+x[1]-x[2]**2+x[2]-x[3]**2
    ce2 = 8*x[1]*(x[1]**2-x[0])-2*(1-x[1])+4*(x[1]-x[2])**2+x[0]**2+x[2]-x[3]**2+x[3]-x[4]**2
    ce3 = 8*x[2]*(x[2]**2-x[1])-2*(1-x[2])+4*(x[2]-x[3])**2+x[1]**2-x[0]+x[3]-x[4]**2+x[0]**2+x[4]-x[5]**2
    ce4 = 8*x[3]*(x[3]**2-x[2])-2*(1-x[3])+4*(x[3]-x[4])**2+x[2]**2-x[1]+x[4]-x[5]**2+x[1]**2+x[5]-x[0]
    ci1 = 8*x[4]*(x[4]**2-x[3])-2*(1-x[4])+4*(x[4]-x[5])**2+x[3]**2-x[2]+x[5]+x[2]**2-x[1]
    ci2 = -(8*x[5] * (x[5]**2-x[4])-2*(1-x[5]) +x[4]**2-x[3]+x[3]**2 - x[4])

    return [obj, ce1,ce2,ce3,ce4,ci1,ci2]


def gradient(x: np.ndarray[float]) -> np.ndarray[float]:
    return pg.estimate_gradient_h(lambda x: fitness(x), x)

@dataclass
class my_minlp:
    # Let's try adding poos to the class until it breaks
    # A dataclass -> Does not brake
    dec_var_updates: DecisionVariablesUpdates
    # A dataframe -> Does not brake
    df: pd.DataFrame
    # A dumped model instance -> Does not brake
    model_dict: dict
    # Malformed Enum -> Does not brake
    initial_state: SolarMedState
    
    dec_var_int_ids: list[str] # Logical / integer decision variables ids
    dec_var_real_ids: list[str]  # Real decision variables ids
    dec_var_model_ids: list[str]
    dec_var_dtypes: list[Type]  # Decision variable data types
    
    dumb_file: str
    
    def __init__(self, dec_var_updates: DecisionVariablesUpdates, model: SolarMED):
        self.dec_var_updates = dec_var_updates
        self.df = pd.DataFrame()
        self.model_dict = model.dump_instance()
        self.initial_state: SolarMedState = model.get_state()
        
        self.dec_var_ids, self.dec_var_dtypes = zip(*[(field.name, get_args(field.type)[0]) for field in fields(DecisionVariables)])
        self.dec_var_int_ids: list[str] = [var_id for var_id, var_type in zip(self.dec_var_ids, self.dec_var_dtypes) if var_type in [bool, int]]
        self.dec_var_real_ids: list[str] = [var_id for var_id, var_type in zip(self.dec_var_ids, self.dec_var_dtypes) if var_type is float]
        # self.dec_var_model_ids: list[str] = [OptimVarIdstoModelVarIdsMapping[var_id].value if var_id in OptimVarIdstoModelVarIdsMapping.__members__ else var_id for var_id in self.dec_var_ids]
        
        with open('dumb_file.txt', 'r') as f:
            self.dumb_file = f.read()
    
    def fitness(self, x: np.ndarray | int) -> np.ndarray[float] | list[float]:
        # print(x)
        
        return fitness(x)
    
    def get_bounds(self) -> tuple[np.ndarray, np.ndarray]:
        """Method that returns the box-bounds of the problem

        Returns:
            tuple[np.ndarray, np.ndarray]: (lower bounds, upper bounds)
        """
        def _get_upper_bounds():
            return [-5]*6
        
        # get_bounds(problem_instance=self)
        return np.array([-5, -5, -5, -5, -5, -5]), np.array([10, 10, 10, 10, 10, 10])
        
        # return (_get_upper_bounds(), [5]*6)
    
    def get_nec(self) -> int:
        return 0
    
    def get_nic(self) -> int:
        return 6
    
    def get_nix(self):
        """ Number of integer decision variables """
        return 2

    def get_name(self) -> str:
        """ Problem’s name """
        return "MINLP"
    
dec_var_updates = DecisionVariablesUpdates(
    sf_active=0,
    ts_active=0,
    med_active=0,
    med_vac_state=0,
    qsf=0,
    qts_src=0,
    qmed_s=0,
    qmed_f=0,
    Tmed_c_out=0,
    Tmed_s_in=0
)

model = SolarMED(
    use_models=True,
    use_finite_state_machine=True,
    resolution_mode='constant-water-props',
    sample_time=400,
    
    # Initial states
    ## Thermal storage
    Tts_h=[50, 50, 50], 
    Tts_c=[50, 50, 50],
    
    ## Solar field
    Tsf_in_ant=np.array([50.0, 50.0, 50.0], dtype=float),
    qsf_ant=np.array([10.0, 10.0, 10.0], dtype=float),
)

prob = pg.problem(my_minlp(dec_var_updates=dec_var_updates, model=model))
print(prob)

algo = pg.algorithm(pg.gaco(gen=50, ker=pop_size))
print(f"Running {algo.get_name()}")
algo.set_verbosity(1) # regulates both screen and log verbosity

# Initialize population and evolve population
pop = pg.population(prob, size=pop_size)
print(f"Initial population: {pop}\nStarting evolution...")
pop = algo.evolve(pop)

# Extract results of evolution
uda=algo.extract(pg.gaco)
print(f"Completed evolution, best fitness: {pop.champion_f[0]}, \nbest decision vector: {pop.champion_x}")

# print(uda.get_log())
for iter_log in uda.get_log():
    print(iter_log)
    
display(pop)
problem = prob.extract(my_minlp)
display(problem.model_dict)
display(problem.initial_state)
display(problem.dumb_file)


Problem name: MINLP
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			2
	Fitness dimension:			7
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	6
	Tolerances on constraints: [0, 0, 0, 0, 0, ... ]
	Lower bounds: [-5, -5, -5, -5, -5, ... ]
	Upper bounds: [10, 10, 10, 10, 10, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

Running GACO: Ant Colony Optimization
Initial population: Problem name: MINLP
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			2
	Fitness dimension:			7
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	6
	Tolerances on constraints: [0, 0, 0, 0, 0, ... ]
	Lower bounds: [-5, -5, -5, -5, -5, ... ]
	Upper bounds: [10, 10, 10, 10, 10, ... ]
	Has bat

Problem name: MINLP
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			2
	Fitness dimension:			7
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	6
	Tolerances on constraints: [0, 0, 0, 0, 0, ... ]
	Lower bounds: [-5, -5, -5, -5, -5, ... ]
	Upper bounds: [10, 10, 10, 10, 10, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 510

	Thread safety: none

Population size: 10

List of individuals: 
#0:
	ID:			17618047876838103197
	Decision vector:	[0.790395, 1.0894, 1.05829, 1.15496, 1, ... ]
	Fitness vector:		[1.5555, 0.0513793, 4.14128, 1.58893, 4.37984, ... ]
#1:
	ID:			6563963847245914411
	Decision vector:	[0.788692, 1.0899, 1.05733, 1.15513, 1, ... ]
	Fitness vector:		[1.55551, 0.0578603, 4.16481, 1.56652, 4.39349, ... ]
#2:
	ID:			4828243030254371842
	Decision vector

{'fixed_model_params': {'med': {'Tmed_s_min': 60,
   'Tmed_s_max': 75,
   'qmed_c_min': 8,
   'qmed_c_max': 21,
   'qmed_s_min': 30,
   'qmed_s_max': 48,
   'qmed_f_min': 5,
   'qmed_f_max': 9},
  'sf': {'nt': 1,
   'npar': 35,
   'ns': 2,
   'Lt': 23.0,
   'Acs': 7.85e-05,
   'Tmax': 110,
   'Tmin': 10,
   'qsf_min': 0.372,
   'qsf_max': 8.928,
   'delay_span': 600},
  'ts': {'qts_src_min': 0.95, 'qts_src_max': 20, 'Tmin': 10, 'Tmax': 120}},
 'use_finite_state_machine': True,
 'sample_time': 400.0,
 'actuators_consumptions': {'med': {'qmed_b': 'med_brine_pump',
   'qmed_f': 'med_feed_pump',
   'qmed_d': 'med_distillate_pump',
   'qmed_c': 'med_cooling_pump',
   'qmed_s': 'med_heatsource_pump'},
  'sf': {'qsf': 'sf_pump'},
  'ts': {'qts_src': 'ts_src_pump'}},
 'Tts_h': array([50., 50., 50.]),
 'Tts_c': array([50., 50., 50.]),
 'Tsf_in': 50.0,
 'Tsf_in_ant': array([50., 50., 50.]),
 'qsf_ant': array([10., 10., 10.]),
 'Tsf_out_ant': 50.0,
 'med_vacuum_state': 0,
 'current_state': <Solar

<SolarMedState.sf_IDLE_ts_IDLE_med_OFF: '000'>

'Más listo que evartisto'

In [3]:
pop


Problem name: MINLP
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			2
	Fitness dimension:			7
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	6
	Tolerances on constraints: [0, 0, 0, 0, 0, ... ]
	Lower bounds: [-5, -5, -5, -5, -5, ... ]
	Upper bounds: [5, 5, 5, 5, 5, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 510

	Thread safety: none

Population size: 10

List of individuals: 
#0:
	ID:			7624351255101974080
	Decision vector:	[-3.35217, -3.20251, -1.82012, -1.66618, -1, ... ]
	Fitness vector:		[1.47932, -11.022, -345.429, -79.2358, -45.6996, ... ]
#1:
	ID:			4990533232948104384
	Decision vector:	[-3.34544, -3.14895, -1.8394, -1.68872, -1, ... ]
	Fitness vector:		[1.49562, -11.0691, -331.7, -80.9484, -48.063, ... ]
#2:
	ID:			7465355501782790829
	Decision ve